In [ ]:
import numpy as np
import proplot as pplt
import fitsio
import glob

%matplotlib inline

In [ ]:
fnames = glob.glob("cat_*", recursive=True)

d = np.concatenate([fitsio.read(f, lower=True) for f in fnames])

In [ ]:
msk = (
    (d["mask_flags"] == 0)
    & (d["flags"] == 0)
    & (d["mdet_s2n"] > 10)
    & (d["mdet_t_ratio"] > 1.2)
    & (d["mfrac"] < 0.1)
)

bin_edges = np.quantile(
    np.hstack([d["psfrec_g_1"][msk], d["psfrec_g_2"][msk]]), 
    np.linspace(0, 1, 16),
)
print(bin_edges)    

In [ ]:
def _bin_stats(bin_edges, binvals, mdet_step, g1, g2):
    binds = np.searchsorted(bin_edges, binvals, side='left')
    binds = np.clip(binds, 0, len(bin_edges)-2)
    
    res = {}
    for col in ["g1", "g2", "g1p", "g1m", "g2p", "g2m"]:
        res[col] = np.zeros(len(bin_edges)-1)
        res["n_%s" % col] = np.zeros(len(bin_edges)-1)
    
    for step in ["noshear", "1p", "1m", "2p", "2m"]:
        msk = mdet_step == step
        if step == "noshear":
            np.add.at(res["g1"], binds[msk], g1[msk])
            np.add.at(res["n_g1"], binds[msk], 1)
            np.add.at(res["g2"], binds[msk], g2[msk])
            np.add.at(res["n_g2"], binds[msk], 1)
        elif step == "1p":
            np.add.at(res["g1p"], binds[msk], g1[msk])
            np.add.at(res["n_g1p"], binds[msk], 1)
        elif step == "1m":
            np.add.at(res["g1m"], binds[msk], g1[msk])
            np.add.at(res["n_g1m"], binds[msk], 1)
        elif step == "2p":
            np.add.at(res["g2p"], binds[msk], g2[msk])
            np.add.at(res["n_g2p"], binds[msk], 1)
        elif step == "2m":
            np.add.at(res["g2m"], binds[msk], g2[msk])
            np.add.at(res["n_g2m"], binds[msk], 1)
    return res

In [ ]:
res = _bin_stats(
    bin_edges, 
    d["psfrec_g_1"][msk], 
    d["mdet_step"][msk], 
    d["mdet_g_1"][msk], 
    d["mdet_g_2"][msk],
)

In [ ]:
R11 = (res["g1p"]/res["n_g1p"] - res["g1m"]/res["n_g1m"])
g1 = res["g1"]/res["n_g1"]

R22 = (res["g2p"]/res["n_g2p"] - res["g2m"]/res["n_g2m"])
g2 = res["g2"]/res["n_g2"]

Rbar = np.mean(R11) + np.mean(R22)

In [ ]:
fig, axs = pplt.subplots()
axs.plot(bin_edges[:-1][res["n_g1"] > 1e1], (g1/Rbar)[res["n_g1"] > 1e1], ".")
axs.plot(bin_edges[:-1][res["n_g2"] > 1e1], (g2/Rbar)[res["n_g2"] > 1e1], ".")

In [ ]:
d.dtype.names

In [ ]:
len(np.unique(d["tilename"]))